# bayesian analysis

In [220]:
import random

In [224]:
results = ['h', 't']

all_coins = {
    'a': {
        'p_heads': 0.5,
        'count': 2,
    },
    'b': {
        'p_heads': 0.6,
        'count': 1,
    },
    'c': {
        'p_heads': 0.9,
        'count': 1,
    },
    'd': {
        'p_heads': 0.3,
        'count': 5,
    },
    'e': {
        'p_heads': 0.8,
        'count': 4,
    },
    'f': {
        'p_heads': 0.7,
        'count': 9,
    },
}


coins = all_coins.keys()

def prob_heads(coin):
    return all_coins[coin]['p_heads']

def prob_tails(coin):
    return 1 - prob_heads(coin)

def prob(result, coin):
    if result == 'h':
        return prob_heads(coin)
    elif result == 't':
        return prob_tails(coin)

def num_coins():
    return sum([all_coins[coin]['count'] for coin in coins])
    
def prob_pick_from_drawer(coin):
    return round(
        float(all_coins[coin]['count']) / num_coins()
    , 4)

def flip(coin):
    r = random.uniform(0,1)
    if r < prob_heads(coin):
        return 'h'
    else:
        return 't'
    
def pick_from_drawer():
    r = random.uniform(0,1)
    idx = 0
    for coin in coins:
        p = prob_pick_from_drawer(coin)
        if r < idx + p:
            return coin
        else:
            idx += p
            
def tot_prob(result):
    return sum([prob(result, coin)*prob_pick_from_drawer(coin) for coin in coins])
            
def update_prior(prior, result):
    
    prior = {
        coin: prob(result, coin) * prior[coin] / tot_prob(result)
        for coin in coins
    }
    
    prior = {
        coin: p/sum(prior.values()) 
        for coin, p in prior.iteritems()
    }

    return prior

In [233]:
# test out our stuff so we know this is implemented correctly
for coin in sorted(coins):
    print 'coin: {}'.format(coin)
    for result in results:
        print '  prob {}: {}'.format(result, prob(result, coin))
    print '  prob drawer: {}'.format(round(prob_pick_from_drawer(coin), 3))
    
# flip each coin 10000x to check results
flips = {coin: 0 for coin in coins}
trials = 10000
for i in range(trials):
    for coin in coins:
        result = flip(coin)
        if result == 'h':
            flips[coin] += 1

print 'simulation frequencies ({} trials)'.format(trials)
for coin in sorted(coins):
    print '  {}: {}'.format(coin, round(float(flips[coin])/trials,3))
    
print 'total probs'
for result in results:
    p = round(tot_prob(result), 3)
    print '  {}: {}'.format(result, p)
print '  h+t: {}'.format(round(sum(tot_prob(result) for result in results)))
print '  drawer: {}'.format(round(sum([prob_pick_from_drawer(coin) for coin in coins]), 3))

coin: a
  prob h: 0.5
  prob t: 0.5
  prob drawer: 0.091
coin: b
  prob h: 0.6
  prob t: 0.4
  prob drawer: 0.045
coin: c
  prob h: 0.9
  prob t: 0.1
  prob drawer: 0.045
coin: d
  prob h: 0.3
  prob t: 0.7
  prob drawer: 0.227
coin: e
  prob h: 0.8
  prob t: 0.2
  prob drawer: 0.182
coin: f
  prob h: 0.7
  prob t: 0.3
  prob drawer: 0.409
simulation frequencies (10000 trials)
  a: 0.503
  b: 0.598
  c: 0.902
  d: 0.295
  e: 0.801
  f: 0.707
total probs
  h: 0.614
  t: 0.386
  h+t: 1.0
  drawer: 1.0


In [236]:
# run experiment!

# establish prior
prior = {coin: prob_pick_from_drawer(coin) for coin in coins}
print 'starting prior'
for coin in sorted(prior.keys()):
    print '  {}: {}'.format(coin, prior[coin])

# pick one coin
new_coin = pick_from_drawer()

trials = 10000
print 'trials: {}'.format(trials)
print 'actual coin: {}'.format(new_coin)

# flip it a bunch of times
flip_results = {result: 0 for result in results}
for i in range(trials):
    result = flip(new_coin)
    flip_results[result] += 1
    prior = update_prior(prior, result)

print 'flip results'
for r, count in flip_results.iteritems():
    print '  {}: {} ({})'.format(r, count, format(round(float(count)/trials, 3)))

print 'resulting prior'
for coin in sorted(prior.keys()):
    print '  {}: {}'.format(coin, round(prior[coin],3))

print 'best guess: {}'.format([c for c,p in prior.iteritems() if p == max(prior.values())][0])

starting prior
  a: 0.0909
  b: 0.0455
  c: 0.0455
  d: 0.2273
  e: 0.1818
  f: 0.4091
trials: 10000
actual coin: d
flip results
  h: 2983 (0.298)
  t: 7017 (0.702)
{'h': 0.2983, 't': 0.7017}
resulting prior
  a: 0.0
  b: 0.0
  c: 0.0
  d: 1.0
  e: 0.0
  f: 0.0
best guess: d
